# Dogs vs Cat Redux
[Dogs vs. Cats Redux: Kernels Edition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

In [ ]:
from importlib import reload
import sys
sys.path.insert(0,'..')
import utils; reload(utils)
from utils import *
from glob import glob
from keras.models import Model

In [ ]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD, rmsprop
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard

In [ ]:
train_dir = '/src/dataset/kaggle/dog_cat_redux/train'
val_dir = '/src/dataset/kaggle/dog_cat_redux/valid'
num_label = 2
batch_size = 64
num_of_epochs = 10
num_training = 23000
num_val = 2000

## Transfer leraning from InceptionV3

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
im_width, im_height = 299, 299 #fixed size for InceptionV3

In [ ]:
# Add new layer after inception model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_label, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze all weight in inceptionV3 for transfer learning
for layer in base_model.layers:
    layer.trainable = False

opt = rmsprop(lr=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    rescale=1/255,
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    rescale=1/255,
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(im_width, im_height),
    batch_size=batch_size,
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(im_width, im_height),
    batch_size=batch_size,
)

In [ ]:
# Plot learning curve on tensor board
tensor_board = TensorBoard(log_dir='/src/tensorboard', batch_size=batch_size)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_generator, 
                    epochs = num_of_epochs,
                    validation_data=val_generator,
                    callbacks = [tensor_board])

## Fine-tune the previous weight

In [ ]:
#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

for i, layer in enumerate(base_model.layers):
    print(i, layer.name)
    
inception_index_top2 = 249
inception_index_top5 = 164

### We chose to train the top 2 inception blocks, i.e. we will freeze the first 249 layers and unfreeze the rest:

In [ ]:
for layer in model.layers[:inception_index_top2]:
      layer.trainable = False
        
for layer in model.layers[inception_index_top2:]:
      layer.trainable = True

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),   
                loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, 
                    #steps_per_epoch = num_training / batch_size,
                    epochs = num_of_epochs,
                    validation_data=val_generator,
                    #validation_steps = num_val / batch_size,
                    callbacks = [tensor_board])